In [ ]:
%pip install srai[all] seaborn lonboard optuna optuna-integration[lightgbm] lightgbm datasets

# Predict Airbnb listing price

In this notebook we will try to predict the price of the AirBnB listings in Paris using geospatial embeddings.

We will create a spatial stratification logic to divide objects to be in different parts of the city.

Later, we will use Optuna to optimize parameters for a given LightGBM model and compare results for multiple embeddings.

<a target="_blank" href="https://colab.research.google.com/github/kraina-ai/srai-tutorial/blob/geopython2025/tutorial/03_airbnb_rental_price_prediction.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Prepare Airbnb dataset

Data from: https://zenodo.org/records/4446043#.ZEV8d-zMI-R

We will download the data from the HuggingFace hub, transform it to a GeoDataFrame and subselect features to make model smaller for tutorial purposes.

In [ ]:
import warnings

import geopandas as gpd
import lightgbm as lgb
import lonboard.basemap
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
from datasets import load_dataset
from h3 import latlng_to_cell
from lonboard import Map, ScatterplotLayer, PolygonLayer
from lonboard.colormap import apply_continuous_cmap
from optuna.integration import LightGBMPruningCallback
from optuna.visualization import (
    plot_contour,
    plot_intermediate_values,
    plot_optimization_history,
    plot_param_importances,
)
from quackosm._osm_tags_filters import merge_osm_tags_filter
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
from srai.embedders import (
    ContextualCountEmbedder,
    CountEmbedder,
    Hex2VecEmbedder,
)
from srai.h3 import h3_to_geoseries, ring_buffer_h3_regions_gdf
from srai.joiners import IntersectionJoiner
from srai.loaders import OSMPbfLoader
from srai.loaders.osm_loaders.filters import GEOFABRIK_LAYERS
from srai.neighbourhoods import H3Neighbourhood

### Precalculate the OpenStreetMap extracts index

In [ ]:
from quackosm.osm_extracts import _get_combined_index

_get_combined_index()

### Load the data from Hugging Face

In [ ]:
ds = load_dataset("kraina/airbnb", "all").filter(
    lambda listing: listing["city"] == "Paris"
)
raw_data = ds["train"].to_pandas()

listings_gdf = gpd.GeoDataFrame(
    raw_data, geometry=gpd.points_from_xy(raw_data.lng, raw_data.lat), crs="EPSG:4326"
)
listings_gdf

In [ ]:
target_column = "realSum"
geometry_column = "geometry"
feature_columns = [
    "person_capacity",
    "cleanliness_rating",
    "guest_satisfaction_overall",
    "bedrooms",
    "room_type",
]
reduced_columns = [
    target_column,
    *feature_columns,
    geometry_column,
]
listings_gdf = listings_gdf[reduced_columns]
listings_gdf

We need to transform categorical feature to one-hot encoding with `get_dummies` from pandas.

In [ ]:
categorical_columns = ["room_type"]

for col in categorical_columns:
    listings_gdf = pd.concat(
        [listings_gdf, pd.get_dummies(listings_gdf[col], prefix=col)], axis=1
    ).drop(col, axis=1)

In [ ]:
listings_gdf.iloc[1].to_dict()

Let's see the distribution of the listings prices.

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

sns.histplot(listings_gdf, x=target_column, kde=True, ax=ax1)
sns.histplot(
    listings_gdf[
        listings_gdf[target_column] <= listings_gdf[target_column].quantile(0.95)
    ],
    x=target_column,
    kde=True,
    ax=ax2,
)

ax1.set_title("Distribution of prices")
ax2.set_title("Zoomed in to <95th percentile")

plt.show()

And listings on the map.

In [ ]:
filtered_listings_gdf = listings_gdf[
    listings_gdf[target_column] <= listings_gdf[target_column].quantile(0.95)
][[geometry_column, target_column]]

layer = ScatterplotLayer.from_geopandas(
    filtered_listings_gdf,
    get_fill_color=apply_continuous_cmap(
        filtered_listings_gdf[target_column]
        / filtered_listings_gdf[target_column].max(),
        mpl.colormaps["RdYlBu_r"],
        alpha=0.4,
    ),
    radius_units="meters",
    radius_min_pixels=1,
    get_radius=filtered_listings_gdf[target_column] / 10,
)

m = Map(
    layer, _height=800, basemap_style=lonboard.basemap.CartoBasemap.DarkMatterNoLabels
)

m

## Split the data spatially

In [ ]:
SPLITTING_H3_RESOLUTION = 8
h3_splitting_column = "h3_splitting"

listings_gdf[h3_splitting_column] = listings_gdf[geometry_column].apply(
    lambda pt: latlng_to_cell(pt.y, pt.x, SPLITTING_H3_RESOLUTION)
)
listings_gdf

### Group the listings statistics per H3 cell

In [ ]:
listing_statistics_per_h3 = listings_gdf.reset_index().groupby(h3_splitting_column).agg(
    {"index": "count", "realSum": "mean"}
).rename(columns={"index": "listings", "realSum": "avg_price"})
listing_statistics_per_h3["quantile"] = pd.qcut(
    listing_statistics_per_h3["avg_price"], 5, labels=False
)
listing_statistics_per_h3

### Split the dataset using these statistics

We will use quantiles to stratify the regions and have both cheap and expensive regions in both groups.

In [ ]:
def split_h3_cells(ratio=0.2, tolerance=1e-2, random_state=42):
    bucket_1 = []
    bucket_2 = []

    for quantile in listing_statistics_per_h3["quantile"].unique():
        _sub_bucket_1 = []
        _sub_bucket_2 = []
        current_listings = listing_statistics_per_h3[
            listing_statistics_per_h3["quantile"] == quantile
        ].sample(
            frac=1, random_state=random_state
        )

        for index, row in current_listings.iterrows():
            current_bucket_2_sum = sum(t[1] for t in _sub_bucket_2)
            current_ratio = current_bucket_2_sum / current_listings["listings"].sum()

            if current_ratio > ratio:
                _sub_bucket_1.append((index, row["listings"]))
                continue

            next_ratio = (current_bucket_2_sum + row["listings"]) / current_listings[
                "listings"
            ].sum()
            if next_ratio > (ratio * (1 + tolerance)):
                _sub_bucket_1.append((index, row["listings"]))
                continue

            if len(_sub_bucket_1) > len(_sub_bucket_2):
                _sub_bucket_2.append((index, row["listings"]))
            else:
                _sub_bucket_1.append((index, row["listings"]))

        bucket_1.extend(_sub_bucket_1)
        bucket_2.extend(_sub_bucket_2)

    bucket_2_total_listings = sum(t[1] for t in bucket_2)
    total_listings = listing_statistics_per_h3["listings"].sum()
    print(
        f"Ratio: {(bucket_2_total_listings / total_listings):.3f} (expected: {ratio})"
    )

    h3_bucket_1 = [t[0] for t in bucket_1]
    h3_bucket_2 = [t[0] for t in bucket_2]

    bucket_1_listings = listing_statistics_per_h3.loc[h3_bucket_1]
    bucket_2_listings = listing_statistics_per_h3.loc[h3_bucket_2]

    bucket_1_mean_value = (
        bucket_1_listings["listings"] * bucket_1_listings["avg_price"]
    ).sum() / bucket_1_listings["listings"].sum()
    bucket_2_mean_value = (
        bucket_2_listings["listings"] * bucket_2_listings["avg_price"]
    ).sum() / bucket_2_listings["listings"].sum()

    print(
        f"Mean listing prices: {(bucket_1_mean_value):.3f} / {(bucket_2_mean_value):.3f}"
    )

    return h3_bucket_1, h3_bucket_2


train_h3_cells, test_h3_cells = split_h3_cells(ratio=0.33, random_state=73)

train_listing_ids = listings_gdf[listings_gdf[h3_splitting_column].isin(train_h3_cells)].index
test_listing_ids = listings_gdf[listings_gdf[h3_splitting_column].isin(test_h3_cells)].index
len(train_listing_ids), len(test_listing_ids)

Display the splitted data on a map.

In [ ]:
m = h3_to_geoseries(
    listings_gdf.loc[train_listing_ids, h3_splitting_column].unique()
).explore(tiles="CartoDB Positron", color="royalblue")
h3_to_geoseries(
    listings_gdf.loc[test_listing_ids, h3_splitting_column].unique()
).explore(m=m, color="orange")
filtered_listings_gdf.explore(target_column, m=m)

## Train the model

Let's prepare some code to train the LigthGBM model and optimize the hyperparameters using Optuna.

It will automatically run a study, evaluate the model trained with best parameters and display some charts with results.

In [ ]:
def find_best_lightgbm_model(
    features, target, previous_best_params=None, n_trials=None, timeout=120
):
    x_train = features.loc[train_listing_ids]
    x_test = features.loc[test_listing_ids]
    y_train = target.loc[train_listing_ids]
    y_test = target.loc[test_listing_ids]

    train_dataset = lgb.Dataset(x_train, label=y_train)
    eval_dataset = lgb.Dataset(x_test, label=y_test)

    # https://github.com/optuna/optuna-examples/blob/main/lightgbm/lightgbm_integration.py
    def objective(trial):
        param = {
            "objective": "regression",
            "metric": ["l1", "l2"],
            "verbosity": -1,
            "boosting_type": "gbdt",
            "feature_pre_filter": False,
            "num_leaves": trial.suggest_int("num_leaves", 2, 256),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        }

        warnings.filterwarnings(
            "ignore",
            category=optuna.exceptions.ExperimentalWarning,
        )

        pruning_callback = LightGBMPruningCallback(trial, "l1")

        gbm = lgb.train(
            param,
            train_dataset,
            valid_sets=[eval_dataset],
            num_boost_round=200,
            callbacks=[
                pruning_callback,
                lgb.early_stopping(stopping_rounds=50, verbose=False),
            ],
        )

        preds = gbm.predict(x_test, num_iteration=gbm.best_iteration + 1)
        pred_labels = np.rint(preds)

        rmse = root_mean_squared_error(y_test, pred_labels)
        return rmse

    optuna.logging.set_verbosity(optuna.logging.WARNING)

    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(
            n_warmup_steps=5, n_min_trials=5, n_startup_trials=5
        ),
        direction="minimize",
    )

    if previous_best_params is not None:
        study.enqueue_trial(previous_best_params)

    study.optimize(
        objective, n_trials=n_trials, timeout=timeout, show_progress_bar=True
    )

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print(f"  Value: {trial.value}")
    print(f"  Params: {trial.params}")

    # Visualize the optimization history.
    plot_optimization_history(study).show()

    # Visualize the learning curves of the trials.
    plot_intermediate_values(study).show()

    # Visualize hyperparameter relationships.
    plot_contour(study).show()

    # Visualize parameter importances.
    plot_param_importances(study).show()

    gbm = lgb.train(
        trial.params,
        train_dataset,
        valid_sets=[eval_dataset],
        num_boost_round=10_000,
        callbacks=[lgb.early_stopping(stopping_rounds=100)],
    )

    y_pred = gbm.predict(x_test)

    result = dict(
        RMSE=root_mean_squared_error(y_test, y_pred),
        R2=r2_score(y_test, y_pred),
        MAE=mean_absolute_error(y_test, y_pred),
        n_features=features.shape[1],
        y_test=y_test,
        y_pred_test=y_pred,
        y_pred_all=gbm.predict(features),
    )

    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    for ax in (ax1, ax2):
        sns.regplot(
            x=y_test,
            y=y_pred,
            scatter_kws=dict(alpha=1, s=2),
            line_kws=dict(color=".2", linestyle="--"),
            ax=ax,
        )
        sns.lineplot(x=[0, y_test.max()], y=[0, y_test.max()], color="red", ax=ax)
        ax.set_xlabel("Actual price")
        ax.set_ylabel("Predicted price")

    q95 = y_test.quantile(0.95)
    ax2.set_xlim(0, q95)
    ax2.set_ylim(0, q95)

    ax1.set_title("Baseline model performance")
    ax2.set_title("Zoomed in to <95th percentile")

    plt.show()

    lgb.plot_importance(gbm, max_num_features=20)
    plt.show()

    print(
        f"RMSE: {result['RMSE']:.2f}, MAE: {result['MAE']:.2f}, R2: {result['R2']:.2f}"
    )

    return result

### Baseline model

In [ ]:
results = []

In [ ]:
features = listings_gdf.drop(
    columns=[target_column, geometry_column, h3_splitting_column]
)
target = listings_gdf[target_column]

features

In [ ]:
optuna_result = find_best_lightgbm_model(features, target, timeout=120)
optuna_result["experiment"] = "baseline"

results.append(optuna_result)

### Count Embedder

In [ ]:
EMBEDDING_H3_RESOLUTION = 10
h3_embedding_column = "h3_embedding"

listings_gdf[h3_embedding_column] = listings_gdf[geometry_column].apply(
    lambda pt: latlng_to_cell(pt.y, pt.x, EMBEDDING_H3_RESOLUTION)
)
listings_gdf

In [ ]:
unique_h3_cells = listings_gdf[h3_embedding_column].unique()
regions_gdf = gpd.GeoDataFrame(
    data=dict(region_id=unique_h3_cells), geometry=h3_to_geoseries(unique_h3_cells)
).set_index("region_id")
regions_gdf

In [ ]:
flat_geofabrik_osm_tags_filter = merge_osm_tags_filter(GEOFABRIK_LAYERS)

In [ ]:
features_gdf = OSMPbfLoader().load(regions_gdf, flat_geofabrik_osm_tags_filter)
features_gdf

In [ ]:
joint = IntersectionJoiner().transform(regions_gdf, features_gdf)
joint

In [ ]:
count_embeddings = CountEmbedder(
    expected_output_features=flat_geofabrik_osm_tags_filter, count_subcategories=True
).transform(regions_gdf, features_gdf, joint)
count_embeddings

In [ ]:
features = listings_gdf.merge(
    count_embeddings.reset_index(),
    left_on=h3_embedding_column,
    right_on=count_embeddings.index.name,
).drop(
    columns=[
        target_column,
        geometry_column,
        h3_splitting_column,
        h3_embedding_column,
        count_embeddings.index.name,
    ]
)
target = listings_gdf[target_column]

features

In [ ]:
optuna_result = find_best_lightgbm_model(features, target, timeout=120)
optuna_result["experiment"] = "count embeddings"

results.append(optuna_result)

### Contextual Count Embedder

In [ ]:
H3_NEIGHBOURHOOD = 5

buffered_regions_gdf = ring_buffer_h3_regions_gdf(
    regions_gdf, distance=H3_NEIGHBOURHOOD
)
buffered_regions_gdf

In [ ]:
features_gdf = OSMPbfLoader().load(buffered_regions_gdf, flat_geofabrik_osm_tags_filter)
features_gdf

In [ ]:
joint = IntersectionJoiner().transform(buffered_regions_gdf, features_gdf)
joint

In [ ]:
context_embeddings = ContextualCountEmbedder(
    count_subcategories=True,
    expected_output_features=flat_geofabrik_osm_tags_filter,
    neighbourhood=H3Neighbourhood(),
    neighbourhood_distance=H3_NEIGHBOURHOOD,
    concatenate_vectors=False,
).transform(buffered_regions_gdf, features_gdf, joint)
context_embeddings

In [ ]:
features = listings_gdf.merge(
    context_embeddings.reset_index(),
    left_on=h3_embedding_column,
    right_on=context_embeddings.index.name,
).drop(
    columns=[
        target_column,
        geometry_column,
        h3_splitting_column,
        h3_embedding_column,
        context_embeddings.index.name,
    ]
)
target = listings_gdf[target_column]

features

In [ ]:
optuna_result = find_best_lightgbm_model(features, target, timeout=120)
optuna_result["experiment"] = "contextual count embeddings"

results.append(optuna_result)

### Hex2Vec Embedder

In [ ]:
features_gdf = OSMPbfLoader().load(buffered_regions_gdf, flat_geofabrik_osm_tags_filter)
joint = IntersectionJoiner().transform(buffered_regions_gdf, features_gdf)
hex2vec_embeddings = Hex2VecEmbedder(
    expected_output_features=flat_geofabrik_osm_tags_filter,
    encoder_sizes=[150, 100, 50, 16],
    count_subcategories=True,
).fit_transform(
    buffered_regions_gdf,
    features_gdf,
    joint,
    neighbourhood=H3Neighbourhood(buffered_regions_gdf),
    batch_size=1024,
    trainer_kwargs=dict(max_epochs=20),
)
features = listings_gdf.merge(
    hex2vec_embeddings.reset_index(),
    left_on=h3_embedding_column,
    right_on=hex2vec_embeddings.index.name,
).drop(
    columns=[
        target_column,
        geometry_column,
        h3_splitting_column,
        h3_embedding_column,
        hex2vec_embeddings.index.name,
    ]
)
target = listings_gdf[target_column]

features

In [ ]:
optuna_result = find_best_lightgbm_model(features, target, timeout=120)
optuna_result["experiment"] = "hex2vec embeddings"

results.append(optuna_result)

## Display results on a map

Now we will display the results for one of the models on the map - both predictions and relative errors.

In [ ]:
aggregated_results = pd.DataFrame(results)[
    ["experiment", "n_features", "RMSE", "R2", "MAE"]
]
aggregated_results

In [ ]:
# results from contextual count embeddings
listings_predictions = listings_gdf.copy()

listings_predictions["prediction"] = results[-2]["y_pred_all"]

# Symmetric mean absolute percentage error
listings_predictions["smape"] = (
    2
    * (listings_predictions[target_column] - listings_predictions["prediction"])
    / (listings_predictions[target_column] + listings_predictions["prediction"])
)

listings_predictions

In [ ]:
filtered_listings_gdf = listings_predictions[
    listings_predictions[target_column] <= listings_predictions[target_column].quantile(0.95)
][[geometry_column, target_column, "prediction"]]

layer = ScatterplotLayer.from_geopandas(
    filtered_listings_gdf,
    get_fill_color=apply_continuous_cmap(
        filtered_listings_gdf["prediction"] / filtered_listings_gdf["prediction"].max(),
        mpl.colormaps["RdYlBu_r"],
        alpha=0.7,
    ),
    radius_units="meters",
    radius_min_pixels=1,
    get_radius=filtered_listings_gdf[target_column] / 10,
)

m = Map(
    layer, _height=800, basemap_style=lonboard.basemap.CartoBasemap.DarkMatterNoLabels
)

m

In [ ]:
filtered_listings_gdf = listings_predictions[
    listings_predictions[target_column] <= listings_predictions[target_column].quantile(0.95)
].copy()

filtered_listings_gdf["normalized_smape"] = (
    filtered_listings_gdf["smape"].apply(
        lambda x, filtered_listings_gdf=filtered_listings_gdf: (
            -x / filtered_listings_gdf["smape"].min()
            if x < 0
            else x / filtered_listings_gdf["smape"].max()
        )
    )
    + 1
) / 2

filtered_listings_gdf["normalized_smape_alpha"] = (
    filtered_listings_gdf["normalized_smape"] - 0.5
).abs() * 2

layer = ScatterplotLayer.from_geopandas(
    filtered_listings_gdf,
    get_fill_color=apply_continuous_cmap(
        filtered_listings_gdf["normalized_smape"],
        mpl.colormaps["bwr_r"],
        alpha=filtered_listings_gdf["normalized_smape_alpha"],
    ),
    radius_units="meters",
    radius_min_pixels=1,
    get_radius=filtered_listings_gdf[target_column] / 10,
)

h3_layer = PolygonLayer.from_geopandas(
    gpd.GeoDataFrame(
        geometry=h3_to_geoseries(
            listings_gdf.loc[test_listing_ids, h3_splitting_column].unique()
        )
    ),
    get_fill_color=[201, 152, 36, 15],
    get_line_color=[201, 152, 36, 150],
    get_line_width=20,
    line_width_min_pixels=1,
    line_width_max_pixels=10,
)

m = Map(
    [h3_layer, layer],
    _height=800,
    basemap_style=lonboard.basemap.CartoBasemap.DarkMatterNoLabels,
)

m


## Task for you

Easy task:
- Add a new feature, showing a distance to the center of a city in meters in a straight line. (You can choose whatever you want as the city centre)
- Train the new model with the embeddings of your choosing.
- Compare the results of new model with previous results.

Hard task:
- Add a new feature, showing a distance to the nearest metro station in meters in a straight line.
- Train the new model with the embeddings of your choosing.
- Compare the results of new model with previous results.

Those two values are present in the original data (columns `dist` and `metro_dist`, so you can compare your results later)

In [ ]:
# Write code here

# You can find answers here: https://github.com/kraina-ai/srai-tutorial/tree/geopython2025/answers